In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
from transformers import TrainingArguments, Trainer

## Loading GPT-2

In [ ]:
# Cargar el modelo y el tokenizer
tokenizer = AutoTokenizer.from_pretrained("datificate/gpt2-small-spanish")
model = AutoModelForCausalLM.from_pretrained("datificate/gpt2-small-spanish")

print("Modelo cargado correctamente.")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

Modelo cargado correctamente.


## Preparing data

In [ ]:
data_path = "data_1.txt"
dataset = load_dataset("text", data_files={"train": data_path})

# Ver un ejemplo del dataset
dataset["train"][:4]

Generating train split: 0 examples [00:00, ? examples/s]

{'text': ['Título: Odio',
  'Artista: Romeo Santos',
  'Sentimientos: negativo, ira, tristeza',
  'yeah yeah envy is sign of admiration hate is the epitome of destruction tal parece amor crece crece desvaneces ofreces amistad excepcion hombre logrado jamas di hogar familia buen porvenir perdon hacerte daño vivo aniquilado despecho reprimido ardiendo celos odio aunque haga feliz intimida entiendo gano guerra hace tiempo olvida perdi you might also like celo besos cuerpo envidia apodera asi quiero aqui odio hombre lado perdi amor pobre diablo amor vivo aniquilado despecho reprimido ardiendo celos odio aunque haga feliz intimida entiendo gano guerra hace tiempo olvida perdi escucha palabras yeah drizzy drake llanto sonrisa turn the lights on desplomo volcan deprisa uh you were the one youve been the one from the moment i seen you sometimes i question if this is all real then i grab on that ass and i firmly believe it and you look like you drawn by an artist you you look like bernice and y

In [ ]:
def tokenize_function(examples):
    # Tokenizar el texto
    tokenized_output = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)
    tokenized_output["labels"] = tokenized_output["input_ids"].copy()
    return tokenized_output

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/8124 [00:00<?, ? examples/s]

## Training GPT-2

In [5]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# Configurar los argumentos de entrenamiento
training_args = TrainingArguments(
    output_dir="./results",  # Carpeta donde guardar los modelos entrenados
    run_name="fine_tuning_gpt2_lyrics",
    overwrite_output_dir=True,  # Sobrescribir salida previa
    num_train_epochs=3,  # Número de épocas
    per_device_train_batch_size=4,  # Tamaño del batch
    save_steps=500,  # Guardar cada 500 pasos
    save_total_limit=2,  # Guardar solo los 2 últimos checkpoints
    logging_dir="./logs", # Directorio para logs
    report_to="none",
    logging_steps=1000,  # Intervalo de logs
    do_train=True,  # Habilitar entrenamiento
    do_eval=False,  # Omitir evaluación
    warmup_steps=100,  # Número de pasos de calentamiento
    learning_rate=5e-5,  # Tasa de aprendizaje
    weight_decay=0.01,  # Decaimiento de pesos
)


In [ ]:
# Crear el objeto Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

# Iniciar el entrenamiento
trainer.train()


Step,Training Loss
1000,0.964300
2000,0.276000
3000,0.217400
4000,0.226800
5000,0.200800
6000,0.207600


TrainOutput(global_step=6093, training_loss=0.34717852412314126, metrics={'train_runtime': 4301.7694, 'train_samples_per_second': 5.666, 'train_steps_per_second': 1.416, 'total_flos': 6368209403904000.0, 'train_loss': 0.34717852412314126, 'epoch': 3.0})

In [ ]:
model.save_pretrained("./gpt2_model_trained_1")
tokenizer.save_pretrained("./gpt2_model_trained_1")

('./gpt2_model_trained_1/tokenizer_config.json',
 './gpt2_model_trained_1/special_tokens_map.json',
 './gpt2_model_trained_1/vocab.json',
 './gpt2_model_trained_1/merges.txt',
 './gpt2_model_trained_1/added_tokens.json',
 './gpt2_model_trained_1/tokenizer.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p /content/drive/MyDrive/gpt2_model_trained_1
!cp -r ./gpt2_model_trained_1 /content/drive/MyDrive/gpt2_model_trained_1

Mounted at /content/drive
cp: cannot stat './logs': No such file or directory
